### Importing libraries and Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
# Loading data 
company = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-01-01'

data = yf.download(company, start=start_date, end=end_date)

prices = data['Close'].values.reshape(-1, 1)

print(f"Data loaded: {len(prices)} days of stock prices.")


C:\Users\HUMAIDU\AppData\Local\Temp\ipykernel_139836\3208946979.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(company, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Data loaded: 1006 days of stock prices.


### Data preprocessing

In [3]:
# Scaling the data to (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(prices)

#Define Training Data (80% of data)
training_data_len = int(len(scaled_data) * 0.8)
train_data = scaled_data[0:training_data_len, :]

# Creating X_train and y_train sequences 
# use a "look back" of 60 days to predict the 61st day
X_train = []
y_train = []
prediction_days = 60

for i in range(prediction_days, len(train_data)):
    X_train.append(train_data[i-prediction_days:i, 0]) # Days 0-59
    y_train.append(train_data[i, 0])

# Convert to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape for LSTM: (Samples, Time Steps, Features)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print(f"Training data prepared: {X_train.shape} samples, each with {X_train.shape[1]} time steps.")


Training data prepared: (744, 60, 1) samples, each with 60 time steps.
